In [2]:
import CoreBGgeneration
from pymongo import MongoClient
import prepare_4_EM
import pandas as pd
import numpy as np
import cPickle as pk
from scipy import sparse, spatial
from collections import OrderedDict
import math
from monary import Monary
from bson.code import Code
import multiprocessing as mp
import bson

In [4]:
def updated_alpha():
    reload(CoreBGgeneration)
    hotbeds_p_corename, hotbeds_p_coreaddr = CoreBGgeneration.loaddistributions("emfeatures_hotbeds_LON","london")
    taap_p_corename, taap_p_coreaddr = CoreBGgeneration.loaddistributions("emfeatures_taap_LON","london")
    hotbeds_lon_params["predicted_ext_id"] = hotbeds_lon_params[["name","address"]].apply(matchrecord,args=(hotbeds_p_corename,hotbeds_p_coreaddr,taap_lon_params[["name","address","mapped_to_ext_id"]],taap_p_corename,taap_p_coreaddr),axis=1)
    result,falses,wrongones = check_result(hotbeds_lon_params,taap_lon_params)

In [5]:
def gen_probvec(name,pCdict,i):
    vec = np.zeros(len(pCdict))
    for word in name:
        idx = pCdict.keys().index(word)
        if len(pCdict[word])==1:
            vec[idx] = pCdict[word][0]
        else:
            vec[idx] = pCdict[word][i]
    
    return vec
    
def genreg_vec(name,pCdict):
    vec=np.zeros(len(pCdict))
    for word in name:
        idx = pCdict.keys().index(word)
        vec[idx]=1.0
        
    return vec

def add_vals_to_vectdic(odict,addict):
    for key,val in addict.iteritems():
        if key in odict:
            odict[key].append(val)
        else:
            odict[key] = []
            odict[key].append(val)
            
    return odict

def makecumulativedict(p_corename1,p_corename2):
    vector_dict = OrderedDict([(k,[]) for k in sorted(p_corename1)])
    vector_dict = add_vals_to_vectdic(vector_dict,p_corename1)
    vector_dict = add_vals_to_vectdic(vector_dict,p_corename2)
    return vector_dict

In [6]:
def check_result(inp1,inp2,idcolumn):
    result = pd.merge(inp1,inp2,left_on=[idcolumn],right_on=["mapped_to_ext_id"])
    falses = result[result[idcolumn]!=result["mapped_to_ext_id_x"]]
    print "Acc: ", 1.0*len(result[result[idcolumn]==result["mapped_to_ext_id_x"]])/len(result)
    print "num falses: ", len(falses)
    print "num total: ", len(result)
    wrongones = pd.merge(falses[["name_x","address_x","mapped_to_ext_id_x",idcolumn]],inp2[["name","address","mapped_to_ext_id"]],left_on=["mapped_to_ext_id_x"],right_on=["mapped_to_ext_id"])
    return result,falses,wrongones

In [7]:
def getColsfromdb(supplier_name):
    client = MongoClient()
    db = client['supplier_static_database']
    map = Code("function(){   for (var key in this) {emit(key,null);} }") 
    reduce = Code("function(key, stuff) {return null;}")
    pT = getattr(db, supplier_name)

    mR = pT.map_reduce(map,reduce,supplier_name + "_keys")
    types_ = [type(v).__name__ for k,v in sorted(pT.find().limit(-1).skip(100).next().items())]
    cols4db = []
    for doc in mR.find():
        cols4db.append(doc["_id"])
    cols4db = sorted(cols4db)
    
    for i,t in enumerate(types_):
        if "unicode" in t:
            types_[i] = "string:50"
        if "ObjectId" in t:
            types_[i] = "id"
        if "list" in t or "NoneType" in t:
            types_[i] = "string:50"
        if "int" in t:
            types_[i]="int64"
        if "float" in t:
            types_[i]="float64"
            
#     print "column length ", len(cols4db)
#     print "type length ", len(types)
    #ks = pT.find().limit(-1).skip(100).next().keys()
    
    try:
        assert len(types_)==len(cols4db)
    except:
        tmpdiff = len(types_)-len(cols4db)
        if tmpdiff<0:
            for i in range(abs(tmpdiff)):
                types_.append("string:10")
        else:
            del cols4db[-tmpdiff:]
            
#     print "column length ", len(cols4db)
#     print "type length ", len(types_)
    return cols4db,types_

    

In [8]:
def getMonaryDF(supplier_name,colnames,types):
    client = Monary()
    data = client.query("supplier_static_database",supplier_name,{},colnames,types)
    dat = pd.DataFrame(np.matrix(data).transpose(), columns=colnames)
    dat.replace("",np.nan,inplace=True)
    dat[["Latitude","Longitude"]] = dat[["Latitude","Longitude"]].replace(0.0,np.nan)
    dat = dat[pd.notnull(dat['mapped_to_ext_id'])]
    #dat.columns = map(unicode,dat.columns)
    return dat

In [30]:
def create_distribution(prop1,prop2,supplier_name1,supplier_name2):
    
    ccolsaddr1 = prepare_4_EM.get_relevcols(prop1,"address")
    ccolsname1 = prepare_4_EM.get_relevcols(prop1,"name")
    
    ccolsaddr2 = prepare_4_EM.get_relevcols(prop2,"address")
    ccolsname2 = prepare_4_EM.get_relevcols(prop2,"name")
    
    output = mp.Queue()
    
    def appendercols(ccols):
        val = dict()
        for c in ccols:
            val[c] = ""
        prop.fillna(value=val)
        prop["address"] = ""
        for c in ccols:
            prop["address"]+=prop[c]
            del prop[c]
            
    if len(ccolsaddr1)>1:
        appendercols(ccolsaddr1)
    if len(ccolsname1)>1:
        appendercols(ccolsname1)
        
    if len(ccolsaddr2)>1:
        appendercols(ccolsaddr2)
    if len(ccolsname2)>1:
        appendercols(ccolsname2)
    
    try:
        params1 = pk.load(open("emfeatures_"+supplier_name1+".pk","rb"))
    except:
        params1 = prepare_4_EM.prepareem(prop1,supplier_name1)
        
    try:
        params2 = pk.load(open("emfeatures_"+supplier_name2+".pk","rb"))
    except:
        params2 = prepare_4_EM.prepareem(prop2,supplier_name2)
    
    citynames1 = params1.city.unique().tolist()
    
    
    citynames2 = params2.city.unique().tolist()
    cities = set(citynames1).intersection(set(citynames2))
    
    alpha = 0.8
    
    processes = [0]*len(cities)
    for i,c in enumerate(cities):
        p1 = params1[params1.city==c]
        p2 = params2[params2.city==c]
        processes[i] = mp.Process(target=city_wide_dedup,args=(p1,p2,c))
        processes[i].start()
        
    for p in processes:
        p.join()
        
    #pCname, pCaddr = city_wide_distr(params[params.city==city_name],0.8,supplier_name+city_name)
    #return params,pCname,pCaddr
    result =[]
    for p in processes:
        result.append(output.get())
    
    return result

In [31]:
def city_wide_dedup(cityparam1,cityparam2,city_name):
    pname1,paddr1 = city_wide_distr(cityparam1,0.8,city_name)
    pname2,paddr2 = city_wide_distr(cityparam2,0.8,city_name)
    
    vector_dict_Pcorename = makecumulativedict(pname1,pname2)
    vector_dict_Pcoreaddr = makecumulativedict(paddr1,paddr2)
    
    cityparam1["final_mapped"]= cityparam1.apply(matchrecord,args=(pname1,paddr1,
                                              cityparam2,
                                              pname2,paddr2,vector_dict_Pcorename,
                                              vector_dict_Pcoreaddr),
                                              axis=1)
    return cityparam1

In [32]:
def DB_dedup(supplier_name1,supplier_name2):
    
    colnames1,types1 = getColsfromdb(supplier_name1)
    colnames2,types2 = getColsfromdb(supplier_name2)
    
    prop1 = getMonaryDF(supplier_name1,colnames1,types1)
    prop2 = getMonaryDF(supplier_name2,colnames2,types2)
    
    #print prop1[[prepare_4_EM.get_relevcols(prop1,"name")[0],prepare_4_EM.get_relevcols(prop1,"address")[0]]]
    netres = create_distribution(prop1,prop2,supplier_name1,supplier_name2)
    
    return pd.concat(netres,ignore_index=True)
        

In [33]:
def city_wide_distr(params,alpha,filename):
    #params = params[params.city==city_name]
    
    
    #Working with only london properties as of now
    names = params.name
    addr = params.address
   
    #Bring out the Core and Background distributions from the EM algorithm of name
    pC_distr_name,pB_distr_name = CoreBGgeneration.core_algo1(names)
    pC_distr_addr, pB_distr_addr = CoreBGgeneration.core_algo1(addr)
    #print "rd prob P(C), P(B) ", pC_distr_addr["rd"], pB_distr_addr["rd"]
    
    #Generate probability of being a core
    pcore_name = CoreBGgeneration.coreprob(pC_distr_name,pB_distr_name,alpha)
    pcore_addr = CoreBGgeneration.coreprob(pC_distr_addr,pB_distr_addr,alpha)
    
    #pk.dump((pcore_name,pcore_addr),open(filename+"distributions.pk","wb"))
    #print "File: ", filename+"distributions.pk created"
    return pcore_name,pcore_addr

In [34]:
def get_cossim(row2,row1,vect_name,vect_addr):#,vect_dict):
#     vec2 = gen_probvec(sent2,vect_dict,1)
#     vec1 = gen_probvec(sent1,vect_dict,0)
#     sim_measure = 1-spatial.distance.cosine(vec1,vec2)
#     if math.isnan(sim_measure):
#                 sim_measure = 0.0

#Before
#     genvec1 = genreg_vec(sent2,vect_dict)
#     genvec2 = genreg_vec(sent1,vect_dict)
    #print row2
    #print row2.loc["address"]
    #print row1.address
    #assert 1==0
    genvec2 = np.concatenate([genreg_vec(row2.loc["name"],vect_name), genreg_vec(row2.loc["address"],vect_addr)])
    genvec1 = np.concatenate([genreg_vec(row1.loc["name"],vect_name), genreg_vec(row1.loc["address"],vect_addr)])
    sim_measure_gen = 1-spatial.distance.cosine(genvec1,genvec2)
    if math.isnan(sim_measure_gen):
                sim_measure_gen = 0.0
            
    #sim_measure+sim_measure_gen
    return sim_measure_gen
    
    

In [35]:
def vect_matchmaker(row, vect_name,vect_addr, data2):
    #Before
    #name_measure = data2.name.apply(get_cossim,args=(row["name"],vect_name))
    #addr_measure = data2.address.apply(get_cossim, args=(row["address"],vect_addr))
    #net_simm = (name_measure)*(addr_measure+0.2*name_measure)
    #return net_simm
     
    measure = data2.apply(get_cossim,args=(row,vect_name,vect_addr),axis=1)
    return measure
    #print "The actual row:\n", row["name"], "\t", row["address"], "\t", row.mapped_to_ext_id
   
    #print "similarity measure", net_simm
    #print data2.loc[net_simm.idxmax(),"mapped_to_ext_id"]
    #return data2.loc[net_simm.idxmax(),"mapped_to_ext_id"]
    
    

In [36]:
def haverdist(row2,row1):
    if pd.isnull(row1.latitude) | pd.isnull(row2.latitude) | pd.isnull(row1.longitude) | pd.isnull(row2.longitude):
        return 0.0
        
    lat1 = math.radians(row1.latitude)
    lon1 = math.radians(row1.longitude)
    lat2 = math.radians(row2.latitude)
    lon2 = math.radians(row2.longitude)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return (1-c)

In [37]:
def distprobs(row,data2):
    dists = data2.apply(haverdist,args=(row,),axis=1)
    return dists

In [38]:
def matching(row2,row1,pCname_self,pCaddr_self, pCsent_name, pCsent_addr):
    name_prob = CoreBGgeneration.dedup(row1.loc["name"],row2.loc["name"],pCsent_name,pCname_self)
    addr_prob = CoreBGgeneration.dedup(row1.loc["address"],row2.loc["address"],pCsent_addr,pCaddr_self)
    #print "name:address probability, ", name_prob, address_prob
    return name_prob*(addr_prob+0.2*name_prob)
    

def matchrecord(row,pCname_self,pCaddr_self,data2,pCname2,pCaddr2,vect_name,vect_addr):
    probs=data2.apply(matching,args=(row,pCname2,pCaddr2,pCname_self,pCaddr_self),axis=1)
    
#     print "The actual row:\n", row
#     print "probability method, the prob: "
#     print probs.nlargest(5)
#     print data2.loc[probs.nlargest(5).index]
    
    vect_prob = vect_matchmaker(row,vect_name,vect_addr,data2)
    
#     print "\nVector method: "
#     print vect_prob.nlargest(5)
#     print data2.loc[vect_prob.nlargest(5).index]

    latlongprob = distprobs(row,data2)
    
    #probs=0.14*probs + 0.86*vect_prob + latlongprob
    fnalprobs = 0.14*probs + 0.86*vect_prob + latlongprob
    
    if (probs + vect_prob).max()<=1e-06:
        return "none"
    
#     print "\nTop probabilities predicted after sum: "
#     print probs.max()
#     print data2.loc[probs.idxmax()]
#     print probs.nlargest(5)
#     print data2.loc[probs.nlargest(5).index]
#     print "endofrow\n"
#     assert 1==0
    return data2.loc[probs.idxmax(),"mapped_to_ext_id"]

In [39]:
%%time
%%capture capt
reload(CoreBGgeneration)
reload(prepare_4_EM)
fnal_df = DB_dedup("hotelbeds_properties","taap_properties")

KeyboardInterrupt: 

CPU times: user 11min 51s, sys: 23min 11s, total: 35min 2s
Wall time: 9h 22s


{25: ({'bch': 4.523067555549233e-09,
   'clb': 0.009963112432858379,
   'countr': 0.8942905342786132,
   'deercreek': 0.8947368421008312,
   'holiday': 0.9999999957979712,
   'hotel': 0.999999999634504,
   'inn': 5.93330576189882e-10,
   'oceanfron': 1.2673320620431735e-07,
   'plantation': 0.8947368421008312,
   'quality': 0.9999999999999999,
   'resor': 0.8947368421008312,
   'rodeway': 0.9999999999973255,
   'sandpebble': 0.9999999873727935,
   'surfside': 1.938582633497882e-08,
   'townhome': 0.8947368421008312,
   'villas': 0.8942905342786132},
  {'1011': 0.9130434694434485,
   '1201': 1.0,
   '1250': 0.9130413136207507,
   '15': 1.0,
   '1550': 0.7777777777711935,
   '1601': 1.0,
   '1650': 0.8399999999948801,
   '17': 1.4320088979698689e-07,
   '1725': 0.9999999999999946,
   'blvd': 9.714198188554879e-08,
   'deercreek': 0.8399999999948801,
   'dr': 0.7777777777711935,
   'fawn': 0.7777777777711935,
   'hwy': 1.4320088979698689e-07,
   'n': 7.11333272097355e-05,
   'ocean': 9.71

In [ ]:
#TESTING AND PRACTICE
#EXTRA

################################
#print hotbeds_lon_params.iloc
# del hotbeds_lon_params["predicted_ext_id"]
# hotbeds_lon_params["predicted_ext_id"] = hotbeds_lon_params[["name","address"]].apply(matchrecord,args=(hotbeds_p_corename,hotbeds_p_coreaddr,
#                                                                                                         taap_lon_params[["name","address","mapped_to_ext_id"]],
#                                                                                                         taap_p_corename,taap_p_coreaddr),axis=1)
# result,falses,wrongones = check_result(hotbeds_lon_params,taap_lon_params)
# taap_lon_params.loc[taap_lon_params["latitude"].idxmax()]["mapped_to_ext_id"]

# updated_alpha()



# pd.merge(result[result["predicted_ext_id"]!=result["mapped_to_ext_id_x"]],taap_lon_params,left_on=["mapped_to_ext_id_x"],right_on=["mapped_to_ext_id"])

#     db = client['supplier_static_database']
#     #prepareem(db.taap_properties, "taap")
    
#     #Get dataframe from mongo
#     prop_subdf= pd.DataFrame(list(db.ean_properties.find()))
#     prop_subdf.columns = map(unicode.lower, prop_subdf.columns)
#     prepareem(prop_subdf,"ean")
    


    #prop_lon_taap[~pd.isnull(prop_lon_taap.mapped_to_ext_id)].to_csv("taap_LON.scsv",sep=";",index=None,encoding="utf-8")
    #prop_lon_hotbed[~pd.isnull(prop_lon_hotbed.mapped_to_ext_id)].to_csv("hotelbeds_LON.scsv",sep=";",index=None,encoding="utf-8")
    

#########################################    
    
    
# client = Monary()
# data = client.query("supplier_static_database","hotelbeds_properties",{},["Name","city"],["type"]*2)
# print type(data)
# types = np.array(np.matrix(data).transpose()[0])[0].tolist()
# bson.BSON.encode(types[0])

# # %%time
# # colnames = ["Name","Address1","Address2","City","Latitude","Longitude","EANHotelID"]
# # types = ["string:72","string:45","string:45","string:20","float64","float64","float64"]
# # ean_params, ean_pcorename,ean_pcore_addr = create_distribution("ean","london",colnames,types)

# client = MongoClient()
# a = "taap_properties"
# db = client["supplier_static_database"]
# b = getattr(db,a)
# db.b

# hbedscolnames = ["Name","address","city","Latitude","Longitude","mapped_to_ext_id"]
# taapcolnames = ["Hotel Name","Street Address 1","City","Latitude","Longitude","mapped_to_ext_id"]
# silverScolnames=["HOTEL NAME","ADDRESS LINE 1","CITY","LATITUDE","LONGITUDE","mapped_to_ext_id"]
# touricocolnames = [""]

####################################


#cols = []
# for m in mR.find():
#     cols.append(m["_id"])
# print cols

# A = [type(v).__name__ for k,v in sorted(db.taap_properties.find_one().items())]
# B = db.taap_properties.find().limit(-1).skip(5).next()
# A[-5:]

# L = pd.DataFrame({"A": np.arange(4),"B":["my","life","",""],"C":["","no","bro",""]})
# L.replace("",np.nan,inplace=True)
# L

# xS= [["holy","crap"],["nahi","aata"],[],np.nan,["A"]]
# bleh = []
# for sub_list in xS:
#     if isinstance(sub_list,float):
#         print "mila nan"
#     else:
#         for item in sub_list:
#             print item
        
# #[np.nan if isinstance(sub_list,float) else item for sub_list in xS for item in sub_list]

###################################################


# from unidecode import unidecode
# from icu import CharsetDetector
# E = "tuff\xc3\xa9-val-de-la-ch\xc3"
# print type(E)
# #E = E.decode('utf-8')
# K= CharsetDetector(E).detect().getName()
# E = E.decode(K)
# print unidecode(E)

# F = "Tuffé Val de la Chéronne"
# F = F.decode('utf-8')
# print unidecode(F)


#######################################################


# pd.DataFrame(np.matrix(data_ean).transpose(), columns=colnames)

# taap= pd.read_csv("taap_LON.scsv",sep=";")
# hotbeds = pd.read_csv("hotelbeds_LON.scsv",sep=";")

# hotbeds_lon_params = prepare_4_EM.prepareem(hotbeds,"hotbeds_LON")
# taap_lon_params = prepare_4_EM.prepareem(taap,"taap_LON")

# hotbeds_lon_params = hotbeds_lon_params[hotbeds_lon_params.city=="london"]
# taap_lon_params = taap_lon_params[taap_lon_params.city=="london"]

# hotbeds_p_corename, hotbeds_p_coreaddr = CoreBGgeneration.loaddistributions("emfeatures_hotbeds_LON","london")
# taap_p_corename, taap_p_coreaddr = CoreBGgeneration.loaddistributions("emfeatures_taap_LON","london")

# result,falses,wrongones = check_result(hotbeds_lon_params,taap_lon_params)

# small_set = pd.DataFrame({"name": falses.name_x.iloc[0:10], "address": falses.address_x.iloc[0:10].tolist(), "mapped": falses.mapped_to_ext_id_x.iloc[0:10]})
# small_set.index = np.arange(10)
# small_set

# small_set.apply(matchrecord,args=(hotbeds_p_corename,hotbeds_p_coreaddr,taap_lon_params[["name","address","mapped_to_ext_id"]],taap_p_corename,taap_p_coreaddr),axis=1)

# taap_lon_params[taap_lon_params["mapped_to_ext_id"]=="h0dhfgwn7f"]


# hotbeds_lon_params[hotbeds_lon_params["mapped_to_ext_id"]=="h0dhfgwn7f"]

# taap_lon_params[taap_lon_params["mapped_to_ext_id"]=="xtvsh8fegw"]

# a = [1,2,3,4]
# reduce((lambda x,y: x*y),a)

# vector_dict_Pcorename = makecumulativedict(hotbeds_p_corename,taap_p_corename)
# vector_dict_Pcoreaddr = makecumulativedict(hotbeds_p_coreaddr,taap_p_coreaddr)



#########################################
# hotbeds_lon_params["vec_predicted"] = hotbeds_lon_params.apply(vect_matchmaker,args=(vector_dict_Pcorename,vector_dict_Pcoreaddr,taap_lon_params),axis=1)

# check_result(hotbeds_lon_params,taap_lon_params,"vec_predicted")

# %%time
# hotbeds_lon_params["final_mapped"] = hotbeds_lon_params.apply(matchrecord,args=(hotbeds_p_corename,hotbeds_p_coreaddr,
#                                                                                 taap_lon_params,
#                                                                                 taap_p_corename,taap_p_coreaddr,vector_dict_Pcorename,
#                                                                                 vector_dict_Pcoreaddr),
#                                                               axis=1)
# fnal_res, fnal_fail, fnal_wrongs = check_result(hotbeds_lon_params,taap_lon_params,"final_mapped")

# fnal_res, fnal_fail, fnal_wrongs = check_result(hotbeds_lon_params,taap_lon_params,"final_mapped")

# pd.merge(fnal_wrongs[["name_x","address_x","name","address","final_mapped"]],taap_lon_params[["name","address","mapped_to_ext_id"]],left_on=["final_mapped"],right_on=["mapped_to_ext_id"],how="left")

# taap_lon_params[taap_lon_params["mapped_to_ext_id"]=="9qg1ebmpuv"]


################################

# test_karo = test_karo.drop(5)
# for idx,row in test_karo.iterrows():
#     matchrecord(row, hotbeds_p_corename,hotbeds_p_coreaddr,
#                                       taap_lon_params[["name","address","mapped_to_ext_id"]],
#                                       taap_p_corename,taap_p_coreaddr,vector_dict_Pcorename,
#                                       vector_dict_Pcoreaddr)
#     assert 1==0

# A = pk.load(open("surfside beachdistributions.pk","rb"))

# hbeds = pk.load(open("hotelbeds_params.pk","rb"))


In [260]:
# %%time
# client = MongoClient()
# db = client['supplier_static_database']
# #db.taap_properties.findOne()
# map = Code("function(){   for (var key in this) {emit(key,null);} }") 
# reduce = Code("function(key, stuff) {return null;}") 
# mR = db.taap_properties.map_reduce(map,reduce,"taap_properties" + "_keys")

# print mR

OperationFailure: ReferenceError: add_to_set is not defined :
@:1:38
